In [5]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 4,
      "metadata": {
        "id": "gi9MbYEGm-Rv"
      },
      "outputs": [],
      "source": [
        "\n",
        "\n",
        "import requests\n",
        "from bs4 import BeautifulSoup\n",
        "import time\n",
        "import csv\n",
        "import random\n",
        "from urllib.parse import urljoin\n",
        "import os"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "BASE_URL = \"http://books.toscrape.com/\"\n",
        "START_URL = BASE_URL\n",
        "OUTPUT_DIR = \"task1_outputs\"\n",
        "OUTPUT_CSV = os.path.join(OUTPUT_DIR, \"books.csv\")\n",
        "USER_AGENT = \"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \"\\\n",
        "             \"(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36\"\n",
        "REQUEST_HEADERS = {\"User-Agent\": USER_AGENT}\n",
        "DELAY_MIN = 1.0\n",
        "DELAY_MAX = 2.5\n",
        "MAX_RETRIES = 3\n",
        "TIMEOUT = 10\n",
        "os.makedirs(OUTPUT_DIR, exist_ok=True)"
      ],
      "metadata": {
        "id": "D_VwcTyNnmpT"
      },
      "execution_count": 8,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "def safe_get(url, session=None, retries=MAX_RETRIES):\n",
        "    s = session or requests.Session()\n",
        "    for attempt in range(1, retries+1):\n",
        "        try:\n",
        "            resp = s.get(url, headers=REQUEST_HEADERS, timeout=TIMEOUT)\n",
        "            resp.raise_for_status()\n",
        "            return resp\n",
        "        except Exception as e:\n",
        "            print(f\"[warn] request failed ({attempt}/{retries}) for {url}: {e}\")\n",
        "            if attempt == retries:\n",
        "                raise\n",
        "            time.sleep(1.5 * attempt)\n",
        "    return None\n",
        "\n",
        "def random_delay():\n",
        "    time.sleep(random.uniform(DELAY_MIN, DELAY_MAX))"
      ],
      "metadata": {
        "id": "NBl9yOLrnstJ"
      },
      "execution_count": 9,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "def parse_book_card(card):\n",
        "\n",
        "\n",
        "    title_tag = card.select_one(\"h3 a\")\n",
        "    title = title_tag[\"title\"].strip() if title_tag and title_tag.has_attr(\"title\") else title_tag.get_text(strip=True)\n",
        "\n",
        "    rel_url = title_tag[\"href\"] if title_tag and title_tag.has_attr(\"href\") else \"\"\n",
        "    product_url = urljoin(BASE_URL, rel_url)\n",
        "\n",
        "    price_tag = card.select_one(\".price_color\")\n",
        "    price = price_tag.get_text(strip=True) if price_tag else \"\"\n",
        "\n",
        "    availability = card.select_one(\".availability\").get_text(strip=True) if card.select_one(\".availability\") else \"\"\n",
        "\n",
        "    rating_class = card.select_one(\"p.star-rating\")\n",
        "    rating = \"\"\n",
        "    if rating_class and rating_class.has_attr(\"class\"):\n",
        "        classes = rating_class[\"class\"]\n",
        "\n",
        "        rating = [c for c in classes if c != \"star-rating\"][0] if len(classes) > 1 else \"\"\n",
        "    return {\n",
        "        \"title\": title,\n",
        "        \"product_url\": product_url,\n",
        "        \"price\": price,\n",
        "        \"availability\": availability,\n",
        "        \"rating\": rating\n",
        "    }\n",
        "\n",
        "def scrape_books(start_url=START_URL, max_pages=None):\n",
        "    results = []\n",
        "    next_page_url = start_url\n",
        "    page_count = 0\n",
        "    session = requests.Session()\n",
        "\n",
        "    while next_page_url:\n",
        "        page_count += 1\n",
        "        print(f\"[info] Fetching page {page_count}: {next_page_url}\")\n",
        "        resp = safe_get(next_page_url, session=session)\n",
        "        soup = BeautifulSoup(resp.text, \"html.parser\")\n",
        "\n",
        "\n",
        "        cards = soup.select(\"article.product_pod\")\n",
        "        for c in cards:\n",
        "            data = parse_book_card(c)\n",
        "            results.append(data)\n",
        "\n",
        "\n",
        "        next_btn = soup.select_one(\"li.next a\")\n",
        "        if next_btn and next_btn.has_attr(\"href\"):\n",
        "            rel_next = next_btn[\"href\"]\n",
        "            next_page_url = urljoin(next_page_url, rel_next)\n",
        "        else:\n",
        "            next_page_url = None\n",
        "\n",
        "        page_count += 0\n",
        "        random_delay()\n",
        "\n",
        "        if max_pages and page_count >= max_pages:\n",
        "            print(\"[info] Reached max_pages limit.\")\n",
        "            break\n",
        "\n",
        "    return results\n"
      ],
      "metadata": {
        "id": "U13uO6gxn6wR"
      },
      "execution_count": 10,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "if __name__ == \"__main__\":\n",
        "    print(\"[start] scraping started\")\n",
        "    try:\n",
        "        data = scrape_books(max_pages=None)  # set an int to limit pages\n",
        "        # Save to CSV\n",
        "        fieldnames = [\"title\", \"product_url\", \"price\", \"availability\", \"rating\"]\n",
        "        with open(OUTPUT_CSV, \"w\", newline=\"\", encoding=\"utf-8\") as f:\n",
        "            writer = csv.DictWriter(f, fieldnames=fieldnames)\n",
        "            writer.writeheader()\n",
        "            for row in data:\n",
        "                writer.writerow(row)\n",
        "        print(f\"[done] saved {len(data)} rows to {OUTPUT_CSV}\")\n",
        "    except Exception as e:\n",
        "        print(f\"[error] scraping aborted: {e}\")\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "zBmf1tZYoZFb",
        "outputId": "043a4def-8fd5-4568-8346-06594494b434"
      },
      "execution_count": 11,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "[start] scraping started\n",
            "[info] Fetching page 1: http://books.toscrape.com/\n",
            "[info] Fetching page 2: http://books.toscrape.com/catalogue/page-2.html\n",
            "[info] Fetching page 3: http://books.toscrape.com/catalogue/page-3.html\n",
            "[info] Fetching page 4: http://books.toscrape.com/catalogue/page-4.html\n",
            "[info] Fetching page 5: http://books.toscrape.com/catalogue/page-5.html\n",
            "[info] Fetching page 6: http://books.toscrape.com/catalogue/page-6.html\n",
            "[info] Fetching page 7: http://books.toscrape.com/catalogue/page-7.html\n",
            "[info] Fetching page 8: http://books.toscrape.com/catalogue/page-8.html\n",
            "[info] Fetching page 9: http://books.toscrape.com/catalogue/page-9.html\n",
            "[info] Fetching page 10: http://books.toscrape.com/catalogue/page-10.html\n",
            "[info] Fetching page 11: http://books.toscrape.com/catalogue/page-11.html\n",
            "[info] Fetching page 12: http://books.toscrape.com/catalogue/page-12.html\n",
            "[info] Fetching page 13: http://books.toscrape.com/catalogue/page-13.html\n",
            "[info] Fetching page 14: http://books.toscrape.com/catalogue/page-14.html\n",
            "[info] Fetching page 15: http://books.toscrape.com/catalogue/page-15.html\n",
            "[info] Fetching page 16: http://books.toscrape.com/catalogue/page-16.html\n",
            "[info] Fetching page 17: http://books.toscrape.com/catalogue/page-17.html\n",
            "[info] Fetching page 18: http://books.toscrape.com/catalogue/page-18.html\n",
            "[info] Fetching page 19: http://books.toscrape.com/catalogue/page-19.html\n",
            "[info] Fetching page 20: http://books.toscrape.com/catalogue/page-20.html\n",
            "[info] Fetching page 21: http://books.toscrape.com/catalogue/page-21.html\n",
            "[info] Fetching page 22: http://books.toscrape.com/catalogue/page-22.html\n",
            "[info] Fetching page 23: http://books.toscrape.com/catalogue/page-23.html\n",
            "[info] Fetching page 24: http://books.toscrape.com/catalogue/page-24.html\n",
            "[info] Fetching page 25: http://books.toscrape.com/catalogue/page-25.html\n",
            "[info] Fetching page 26: http://books.toscrape.com/catalogue/page-26.html\n",
            "[info] Fetching page 27: http://books.toscrape.com/catalogue/page-27.html\n",
            "[info] Fetching page 28: http://books.toscrape.com/catalogue/page-28.html\n",
            "[info] Fetching page 29: http://books.toscrape.com/catalogue/page-29.html\n",
            "[info] Fetching page 30: http://books.toscrape.com/catalogue/page-30.html\n",
            "[info] Fetching page 31: http://books.toscrape.com/catalogue/page-31.html\n",
            "[info] Fetching page 32: http://books.toscrape.com/catalogue/page-32.html\n",
            "[info] Fetching page 33: http://books.toscrape.com/catalogue/page-33.html\n",
            "[info] Fetching page 34: http://books.toscrape.com/catalogue/page-34.html\n",
            "[info] Fetching page 35: http://books.toscrape.com/catalogue/page-35.html\n",
            "[info] Fetching page 36: http://books.toscrape.com/catalogue/page-36.html\n",
            "[info] Fetching page 37: http://books.toscrape.com/catalogue/page-37.html\n",
            "[info] Fetching page 38: http://books.toscrape.com/catalogue/page-38.html\n",
            "[info] Fetching page 39: http://books.toscrape.com/catalogue/page-39.html\n",
            "[info] Fetching page 40: http://books.toscrape.com/catalogue/page-40.html\n",
            "[info] Fetching page 41: http://books.toscrape.com/catalogue/page-41.html\n",
            "[info] Fetching page 42: http://books.toscrape.com/catalogue/page-42.html\n",
            "[info] Fetching page 43: http://books.toscrape.com/catalogue/page-43.html\n",
            "[info] Fetching page 44: http://books.toscrape.com/catalogue/page-44.html\n",
            "[info] Fetching page 45: http://books.toscrape.com/catalogue/page-45.html\n",
            "[info] Fetching page 46: http://books.toscrape.com/catalogue/page-46.html\n",
            "[info] Fetching page 47: http://books.toscrape.com/catalogue/page-47.html\n",
            "[info] Fetching page 48: http://books.toscrape.com/catalogue/page-48.html\n",
            "[info] Fetching page 49: http://books.toscrape.com/catalogue/page-49.html\n",
            "[info] Fetching page 50: http://books.toscrape.com/catalogue/page-50.html\n",
            "[done] saved 1000 rows to task1_outputs/books.csv\n"
          ]
        }
      ]
    }
  ]
}


{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'provenance': []},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'}},
 'cells': [{'cell_type': 'code',
   'execution_count': 4,
   'metadata': {'id': 'gi9MbYEGm-Rv'},
   'outputs': [],
   'source': ['\n',
    '\n',
    'import requests\n',
    'from bs4 import BeautifulSoup\n',
    'import time\n',
    'import csv\n',
    'import random\n',
    'from urllib.parse import urljoin\n',
    'import os']},
  {'cell_type': 'code',
   'source': ['BASE_URL = "http://books.toscrape.com/"\n',
    'START_URL = BASE_URL\n',
    'OUTPUT_DIR = "task1_outputs"\n',
    'OUTPUT_CSV = os.path.join(OUTPUT_DIR, "books.csv")\n',
    'USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "\\\n',
    '             "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"\n',
    'REQUEST_HEADERS = {"User-Agent": USER_AGENT}\n',
    'DELAY_MIN = 1.0\n',
    'DELAY_MAX = 2.5\n',
  